In [2]:
import numpy as np
from PIL import Image, ImageEnhance
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Polygon, Circle
import rasterio
from rasterio.windows import Window
import random
import pickle
from shapely import geometry
from matplotlib import colors as mcolors
from random import shuffle

### 1. Get The Satellite Image Files

In [3]:
feb9_dir = 'files/S2 2-9-22/GRANULE/L1C_T40QFM_A025743_20220209T064918/IMG_DATA/'
feb_19_dir = 'files/S2 2-19-22/GRANULE/L1C_T40QFM_A025886_20220219T064947/IMG_DATA/'

bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12']

### Load Bands

In [4]:
# LOAD BANDS

bandset = []
bandpaths = []

def load_bands(dir):
    global bandset, bandpaths
    bandset = []
    bandpaths = []
    for file in os.listdir(dir):
        if file.endswith(".jp2") and file.split('.jp2')[0].split('_')[-1] in bands:
            bandpaths.append(dir + file)
            # bandset.append(rasterio.open(dir + file))
    bandpaths.sort()
    for band in bandpaths:
        bandset.append(rasterio.open(band))

load_bands(feb9_dir)

In [5]:
# DISPLAY THE BANDS

%matplotlib qt
def show_bands(skip=4):
    f, axarr = plt.subplots(3,4)
    for ind, band in enumerate(bandset):
        axarr[ind//4][ind%4].imshow(band.read(1)[::skip, ::skip])
show_bands()

: 

### Preprocess The Bands

In [5]:
# GET THE CROP AREA

fig, ax = plt.subplots()
img = bandset[0]
print(img.shape)
plt.imshow(img.read(1))

tl = []
br = []
tlcoords = []
brcoords = []

def onclick(event):
    global tl, br, tlcoords, brcoords
    x, y = event.xdata, event.ydata
    rightclick = event.button == 3
    if rightclick:
        br = [x / img.shape[1], y / img.shape[0]]
        brcoords = [round(x), round(y)]
        print('bottom right', brcoords)
    else:
        tl = [x / img.shape[1], y / img.shape[0]]
        tlcoords = [round(x), round(y)]
        print('top left', tlcoords)
    if tl and br:
        ax.add_patch(Rectangle((tlcoords[0], tlcoords[1]), brcoords[0] - tlcoords[0], brcoords[1] - tlcoords[1], fill=False, edgecolor='red'))
        plt.draw()

cid = fig.canvas.mpl_connect('button_press_event', onclick)

(10980, 10980)
top left [2994, 7973]
bottom right [7153, 9607]


In [157]:
# CLIP EACH BAND TO THE CROP AREA

clipFolder = 'clipped/'
clipname = 'clip-'
for path in bandpaths:
    with rasterio.open(path) as src:
        # Create a Window and calculate the transform from the source dataset
        wid = src.shape[1]
        hi = src.shape[0]
        xsize = round((br[0] - tl[0]) * wid)
        ysize = round((br[1] - tl[1]) * hi)
        window = Window(round(tl[0] * wid), round(tl[1] * hi), round(xsize * wid), round(ysize * hi))
        transform = src.window_transform(window)
        profile = src.profile
        profile.update({
            'height': ysize,
            'width': xsize,
            'transform': transform
        })
        
        with rasterio.open(clipFolder + clipname +  path.split('/')[-1], 'w', **profile) as dst:
            # Read the data from the window and write it to the output raster
            dst.write(src.read(window=window))
            
        print(clipFolder + clipname + path.split('/')[-1])

clipped/clip-T40QFM_20220209T064019_B02.jp2
clipped/clip-T40QFM_20220209T064019_B03.jp2
clipped/clip-T40QFM_20220209T064019_B04.jp2
clipped/clip-T40QFM_20220209T064019_B05.jp2
clipped/clip-T40QFM_20220209T064019_B06.jp2
clipped/clip-T40QFM_20220209T064019_B07.jp2
clipped/clip-T40QFM_20220209T064019_B08.jp2
clipped/clip-T40QFM_20220209T064019_B11.jp2
clipped/clip-T40QFM_20220209T064019_B12.jp2
clipped/clip-T40QFM_20220209T064019_B8A.jp2


In [16]:
# USE CLIPPED BANDS

load_bands('clipped/')
show_bands()

### Mark Areas Of Interest

#### Combine Bands To RGB For Preview

In [23]:
#            r  g  b
bandorder = [2, 1, 0]

def combineLayers(bandset, bandorder):
    img = []
    for band in bandorder:
        img.append(bandset[band].read(1))

    maxwid = max(img, key=lambda x: x.shape[1]).shape[1]
    maxhi = max(img, key=lambda x: x.shape[0]).shape[0]

    # scale smaller images to the same size
    for ind, band in enumerate(img):
        if band.shape[0] < maxhi:
            img[ind] = np.repeat(band, maxhi // band.shape[0], axis=0)
        if band.shape[1] < maxwid:
            img[ind] = np.repeat(img[ind], maxwid // band.shape[1], axis=1)

    # in case scaling didn't go the full way, add 0's to pad
    for ind, band in enumerate(img):
        if band.shape[0] < maxhi:
            img[ind] = np.vstack([band, np.repeat(np.zeros(maxwid), maxhi - band.shape[0], axis=0)])
        if band.shape[1] < maxwid:
            img[ind] = np.hstack([band, np.repeat(np.zeros((maxhi,1)), maxwid - band.shape[1], axis=1)])

    return np.array(img), maxhi, maxwid

def getRGBImage(bandorder):
    # load each into an array
    img, maxhi, maxwid = combineLayers(bandset, bandorder)
    print(maxhi, maxwid, img.shape)

    img = img / 32

    # make sure that the whites that are too bright don't overflow
    bounds = lambda x: max(0, min(x, 255))
    vfunc = np.vectorize(bounds)
    img = vfunc(img)
    print(np.max(img))
    img = img.astype(np.uint8)

    rgbArray = np.zeros((maxhi, maxwid, 3), 'uint8')
    rgbArray[:,:,0] = img[0]
    rgbArray[:,:,1] = img[1]
    rgbArray[:,:,2] = img[2]

    return rgbArray

preview = getRGBImage(bandorder)
plt.imshow(preview)

1782 3773 (3, 1782, 3773)
255.0


#### Select ROIs

In [18]:
# holds the polygons for each class, defined separately so classes can be added when run again
areas = {}

In [17]:
macroclasses = ['Built-Up', 'Water', 'Clouds', "Ground"]
classes = {
    "Built-Up": ["buildings", "roads"],
    "Clouds": ["clouds", "shadow"],
    "Water": ["ocean"],
    "Ground": ["vegetation", "bare_soil", "mountain"]
}
colors = {
    "Built-Up": "red",
    "Water": "blue",
    "Clouds": "white",
    "Ground": "peachpuff",
    "clouds": "white",
    "shadow": "black",
    "roads": "gray",
    "buildings": "firebrick",
    "ocean": 'blue',
    "vegetation": "green",
    "bare_soil": "sandybrown",
    "mountain": "darkgoldenrod"
}
# reverse map colors to classes
colorMap = {}
for key in classes:
    for val in classes[key]:
        rgb = mcolors.to_rgb(colors[val])[:3]

        colorMap[tuple(np.round(np.array(rgb)*255).astype(np.uint8).tolist())] = val

In [10]:
# GET THE CLASSIFICATION AREAS

fig, ax = plt.subplots()
ax.imshow(preview)

cur_macroclass = 0
cur_class = 0

coords = []             # normalized coordinates because some images are larger than others
pixelcoords = []        # coordinates in pixels, relative to the reference image

currentcircles = []     # holds the circles drawn on the image

pressingShift = False

curtext = None
curpolygon = None

mcstr = macroclasses[cur_macroclass] # macro class name
curtext = plt.text(0, 0.94, mcstr + ',' + classes[mcstr][cur_class], color='red', transform = ax.transAxes)
fig.canvas.draw()

def onclick(event):
    if not pressingShift: return
    global coords, currentcircles, pixelcoords, curpolygon
    x, y = event.xdata, event.ydata
    rightclick = event.button == 3

    mcname = macroclasses[cur_macroclass]
    cname = classes[mcname][cur_class]
    
    # close the polygon, reset coordinates
    if rightclick:
        # add the coordinates to the areas
        
        if not mcname in areas:
            areas[mcname] = []
        if not cname in areas:
            areas[cname] = []
        
        areas[mcname].append(coords)
        areas[cname].append(coords)
        

        coords = []
        pixelcoords = []
        for i in currentcircles:
            i.remove()
        fig.canvas.draw()
        currentcircles = []
        curpolygon = None
    # add coordinate, draw point
    else:
        coords.append((x / preview.shape[1], y / preview.shape[0]))
        pixelcoords.append((round(x), round(y)))
        circle = Circle((round(x), round(y)), radius=3, fill=True, alpha=0.8, color='magenta')
        ax.add_patch(circle)
        currentcircles.append(circle)
    
    # draw a polygon if more than 3 coordinates
    if len(coords) > 2:
        if curpolygon: curpolygon.remove()
        curpolygon = Polygon(np.array(pixelcoords), fill=True, alpha=0.6, color=colors[cname])
        ax.add_patch(curpolygon)
    
    fig.canvas.draw()

# detect if the shift key is pressed
def onKeyPress(event):
    global pressingShift
    if event.key == 'shift':
        pressingShift = True
# detect if the shift key is released, detect if 1, 2, 3, or 4 are pressed
def onKeyUp(event):
    global pressingShift, cur_class, cur_macroclass, curtext
    if event.key == 'shift':
        pressingShift = False
    if event.key == '1':
        cur_macroclass = max(0, cur_macroclass - 1)
        cur_class = 0
    if event.key == '2':
        cur_macroclass = min(len(macroclasses) - 1, cur_macroclass + 1)
        cur_class = 0
    if event.key == '3':
        cur_class = max(0, cur_class - 1)
    if event.key == '4':
        cur_class = min(len(classes[macroclasses[cur_macroclass]]) - 1, cur_class + 1)

    mcstr = macroclasses[cur_macroclass]
    if curtext: curtext.remove()
    curtext = plt.text(0, 0.94, mcstr + ',' + classes[mcstr][cur_class], color='red', transform = ax.transAxes)
    fig.canvas.draw()

cid = fig.canvas.mpl_connect('button_press_event', onclick)
keydown = fig.canvas.mpl_connect('key_press_event', onKeyPress)
keyup = fig.canvas.mpl_connect('key_release_event', onKeyUp)

#### Save The Training Areas

In [173]:

pickle.dump(areas, open("training.p", "wb"))

In [172]:
areas.keys()

dict_keys(['Built-Up', 'buildings', 'roads', 'Water', 'ocean', 'Clouds', 'clouds', 'Ground', 'vegetation', 'bare_soil', 'mountain', 'shadow'])

In [19]:
areas = pickle.load(open("training.p", "rb"))

### Train The Classification Model Using The ROIs And Classify The Image

#### Set Up A Classifier

In [20]:
class Classifier:
    def __init__(self, bandset):
        self.classes = {}
        self.bandset = bandset
        self.bigimg = None

        ret, maxhi, maxwid = combineLayers(self.bandset, range(len(self.bandset)))

        self.bigimg = np.zeros((maxhi, maxwid, len(self.bandset))) # array with all bands (w x h x #bands)
        for y in range(maxhi):
            for x in range(maxwid):
                self.bigimg[y,x] = ret[:,y,x]
    
    def save_training(self, filename):
        pickle.dump(self.classes, open(filename, "wb"))
    
    def load_training(self, data):
        self.classes = pickle.load(open(data, "rb"))

    # add a class to the classifier
    def add(self, name, polygons):
        avgvalues = []
        # go through each band in the bandset, read it, and get the average value for every polygon through that band
        for bandimg in self.bandset:
            band = bandimg.read(1)
            averages = []
            for polygon in polygons:
                points = []
                for point in polygon:
                    # all the points are normalized, so scale them back up
                    points.append((round(point[0] * band.shape[1]), round(point[1] * band.shape[0])))
                avg = self.getAverageColor(band, points)
                averages.append(avg)
            avgvalues.append(np.average(averages, axis=0))
        self.classes[name] = np.array(avgvalues)

    def getBoundingSquare(self, points):
        xs = [p[0] for p in points]
        ys = [p[1] for p in points]
        xmin = min(xs)
        xmax = max(xs)
        ymin = min(ys)
        ymax = max(ys)
        return (xmin, xmax, ymin, ymax)

    # gets a square of side length radius, centered at the point
    def getNearbyPixels(self, x, y, radius=2):
        xmin = max(0, x - radius)
        xmax = min(self.bigimg.shape[1], x + radius)
        ymin = max(0, y - radius)
        ymax = min(self.bigimg.shape[0], y + radius)
        pixels = self.bigimg[ymin:ymax, xmin:xmax].flatten().tolist()
        # pad with zeroes if not long enough
        if len(pixels) != (2 * radius + 1) ** 2 * self.bigimg.shape[2]:
            pads = ((2 * radius + 1) ** 2 - len(pixels))
            pixels.extend([[0]*13 for _ in range(pads)])
        return pixels

    def getPixelsWithinPolygon(self, img, points):
        xmin, xmax, ymin, ymax = self.getBoundingSquare(points)
        polygon = geometry.polygon.Polygon(points)
        pixels = []
        for x in range(xmin, xmax + 1):
            for y in range(ymin, ymax + 1):
                if polygon.contains(geometry.Point(x, y)):
                    pixels.append(img[y][x])
        return pixels
    
    # average pixel color within a region
    def getAverageColor(self, img, points):
        pixels = self.getPixelsWithinPolygon(img, points)
        return np.average(pixels, axis=0)
    
    # get coordinates of pixels that are within a polygon
    def getPixelCoords(self, points):
        xmin, xmax, ymin, ymax = self.getBoundingSquare(points)
        polygon = geometry.polygon.Polygon(points)
        pixels = []
        for x in range(xmin, xmax + 1):
            for y in range(ymin, ymax + 1):
                if polygon.contains(geometry.Point(x, y)):
                    pixels.append(self.getNearbyPixels(x, y))
        return pixels

    # very simple spectral analysis and mapping
    def nearestClass(self, x, y, radius=2):
        if radius > 0:
            xmin = max(0, x - radius)
            xmax = min(self.bigimg.shape[1], x + radius)
            ymin = max(0, y - radius)
            ymax = min(self.bigimg.shape[0], y + radius)
            pixel = np.average(self.bigimg[ymin:ymax, xmin:xmax], axis=0)
        else:
            pixel = self.bigimg[y][x]
        mindist = float('inf')
        minclass = None
        for name, values in self.classes.items():
            dist = np.linalg.norm(pixel - values)
            if dist < mindist:
                mindist = dist
                minclass = name
        return minclass
    
    def classifyRegion(self, x1, y1, x2, y2, verbose=False, radius=2):
        results = np.zeros((y2 - y1, x2 - x1, 3), 'uint8')
        possibleColors = {}
        for key in colors:
            possibleColors[key] = np.round(np.array(mcolors.to_rgb(colors[key])[:3]) * 255).astype('uint8')

        for y in range(y1, y2):
            for x in range(x1, x2):
                classname = self.nearestClass(x, y, radius)
                results[y-y1][x-x1] = possibleColors[classname]
            if verbose:
                if round((y - y1) / (y2 - y1) * 100) % 10 == 0:
                    print(str(round((y - y1) / (y2 - y1) * 100)) + '%')
        
        return np.array(results)
    
    def classify(self, verbose=False, radius=2):
        return self.classifyRegion(0, 0, self.bigimg.shape[1], self.bigimg.shape[0], verbose, radius)
        
c = Classifier(bandset)

In [21]:
# c.load_training('classifier1.p')
for key in classes:
    for x in classes[key]:
        if x not in c.classes:
            c.add(x, areas[x])
            print(x)
print('done adding data!')
c.save_training('classifier1.p')

buildings
roads
clouds
shadow
ocean
vegetation
bare_soil
mountain
done adding data!


#### Preview And Render Classification

In [24]:
fig, ax = plt.subplots()
ax.imshow(preview)

def previewOnclick(event):
    x, y = event.xdata, event.ydata
    xmin = max(0, int(x - 200))
    xmax = min(preview.shape[1], int(x + 200))
    ymin = max(0, int(y - 200))
    ymax = min(preview.shape[0], int(y + 200))
    classification = c.classifyRegion(xmin, ymin, xmax, ymax, verbose=True, radius=1)
    print(classification.shape)
    ax.imshow(classification)
    fig.canvas.draw()

cid = fig.canvas.mpl_connect('button_press_event', previewOnclick)


0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
(400, 400, 3)


In [25]:
classification = c.classify(verbose=True, radius=1)
render = Image.fromarray(classification, mode='RGB')
render.show()
pickle.dump(classification, open('classification.p', 'wb'))

0%
50%


In [26]:
render.show()

### Perform An Accuracy Assessment

In [27]:
previewimg = Image.fromarray(preview, mode='RGB')
addColor = ImageEnhance.Color(previewimg)
pimg2 = addColor.enhance(2)
brighten = ImageEnhance.Brightness(pimg2)
pimg2 = brighten.enhance(1.5)
preview = np.array(pimg2)

#### Get 10 Random Test Points For Each Class

In [28]:
# record all the pixel locations of every color
colorPixels = {}
for y in range(len(classification)):
    for x in range(len(classification[y])):
        color = tuple(classification[y][x])
        if color not in colorPixels:
            colorPixels[color] = []
        colorPixels[color].append((x, y))

testpoints = []
for key in colorPixels:
    points = random.choices(colorPixels[key], k=10)
    testpoints.append(points)

#### Go Trough Each Test Point And Manually Classsify

In [33]:
fig, axarr = plt.subplots()
axarr.imshow(preview)
# axarr[1].imshow(classification)

curclass = 0
curpoint = 0
curtext = None
cursquare = None

failed = 0
passed = 0
perClass = {}


point = testpoints[curclass][curpoint]
predicted = tuple(classification[point[1]][point[0]].tolist())
colorname = colorMap[predicted]

cursquare = axarr.add_patch(Rectangle((point[0] - 3, point[1] - 3), 6, 6, edgecolor='red', facecolor='none'))
curtext = plt.text(0, 0.94, colorname, color='red', transform = ax.transAxes)

radius = 300

# y axis goes from top down, so max and min are reversed
axarr.axis([
            max(0, point[0]-radius),                  # x min
            min(preview.shape[1], point[0]+radius),   # x max
            min(preview.shape[0], point[1]+radius),   # y max
            max(0, point[1]-radius)                   # y min
            ])

perClass[colorname] = { 'passed': 0, 'failed': 0 }

def accuracyKeyPress(event):
    global cursquare, curtext, passed, failed, curpoint, curclass, curtext, colorname
    if colorname not in perClass:
        perClass[colorname] = {
            'passed': 0,
            'failed': 0
        }
    # failed
    if event.key == '1':
        failed += 1
        perClass[colorname]['failed'] += 1
        curpoint += 1
        print('failed', end=' ')
    
    # passed
    if event.key == '2':
        passed += 1
        perClass[colorname]['passed'] += 1
        curpoint += 1
        print('passed', end=' ')
    
    
    if curpoint >= len(testpoints[curclass]):
        curclass += 1
        curpoint = 0
        print()
    
    # define new point and color, remove old markers before replacing them
    point = testpoints[curclass][curpoint]
    predicted = tuple(classification[point[1]][point[0]].tolist())
    colorname = colorMap[predicted]
    cursquare.remove()
    curtext.remove()
    cursquare = axarr.add_patch(Rectangle((point[0] - 3, point[1] - 3), 6, 6, edgecolor='red', facecolor='none'))
    curtext = plt.text(0, 0.94, colorname, color='red', transform = ax.transAxes)
    
    axarr.axis([max(0, point[0]-radius), min(preview.shape[1], point[0]+radius), min(preview.shape[0], point[1]+radius), max(0, point[1]-radius)])
    fig.canvas.draw()

keydown = fig.canvas.mpl_connect('key_press_event', accuracyKeyPress)

passed passed passed passed passed passed passed passed passed passed 
failed failed failed failed passed failed failed failed failed failed 
passed passed failed passed passed passed passed passed passed passed 
failed passed failed failed failed failed failed failed failed failed 
failed failed passed failed passed passed failed failed failed passed 
passed failed passed failed failed failed failed failed passed passed 
failed passed passed passed passed failed failed passed passed passed 
passed passed passed passed passed passed passed failed passed passed 


Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/matplotlib/cbook/__init__.py", line 287, in process
    func(*args, **kwargs)
  File "/var/folders/bw/4tzf4sn540g21cjq2sgsty4w0000gn/T/ipykernel_44111/3544117365.py", line 62, in accuracyKeyPress
    point = testpoints[curclass][curpoint]
IndexError: list index out of range


In [34]:
print(str(100*passed / (passed + failed)) + "% Accuracy")
for key in perClass:
    print(key + ': ' + str(100*perClass[key]['passed'] / 10) + "% Accuracy")


56.25% Accuracy
ocean: 100.0% Accuracy
shadow: 10.0% Accuracy
mountain: 90.0% Accuracy
roads: 10.0% Accuracy
bare_soil: 40.0% Accuracy
buildings: 40.0% Accuracy
vegetation: 70.0% Accuracy
clouds: 90.0% Accuracy


### Derive Metrics: Vegetation Cover, Soil Exposed

In [35]:
def getCover(img, lookfor, filters): # returns percent cover of a region in an image or region, filtering out unwanted regions
    wanted = []
    unwanted = []
    for i in lookfor:
        wanted.append(tuple(np.round(np.array(mcolors.to_rgb(colors[i])[:3])*255).tolist()))
    for i in filters:
        unwanted.append(tuple(np.round(np.array(mcolors.to_rgb(colors[i])[:3])*255).tolist()))
    
    land_pixels = 0
    wanted_pixels = 0

    for y in range(len(img)):
        for x in range(len(img[y])):
            p = tuple(img[y,x].tolist())
            if p in unwanted:
                continue
            land_pixels += 1
            if p in wanted:
                wanted_pixels += 1
    
    return 100 * wanted_pixels / land_pixels

def getVegetationCover(img): 
    return getCover(img, ['vegetation'], ['ocean'])

def getSoilCover(img):
    return getCover(img, ['bare_soil', 'mountain'], ['ocean'])

print(getVegetationCover(classification))
print(getSoilCover(classification))

1.2074548808237282
50.74261897895081
